In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import requests
import time
from sqlalchemy import inspect, create_engine
import matplotlib.pyplot as plt
import numpy as np


In [2]:
#Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

Navigate to browser location for data extraction

In [3]:
#Website searching Data Analyst positions
indeed = "https://www.indeed.com/jobs?q=data+analyst"

browser.visit(indeed)

In [4]:
html = browser.html
html[0:500]

'<html dir="ltr" lang="en" class="js-focus-visible" data-js-focus-visible=""><head>\n    <link rel="shortcut icon" href="/images/favicon.ico">\n    <title>Data Analyst Jobs, Employment | Indeed.com</title>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8">\n    <meta name="description" content="28,441 Data Analyst jobs available on Indeed.com. Apply to Data Analyst, Analyst, Entry Level Data Analyst and more!">\n    <meta name="keywords" content="Data Analyst Jobs, Employment,  c'

In [5]:
soup= BeautifulSoup(browser.html, 'lxml')
type(soup)

bs4.BeautifulSoup

In [6]:
#Find code to loop through web page and pull the title of the job and possibly the location, then find code to click the job and acquire the salary.
jobs = soup.find_all('div', {'class': 'cardOutline'})
len(jobs)

15

Loop code to run iterations

In [7]:
#Create for loops 
rows = []
browser.visit(indeed)
for i in range(1, 50):
    
    html = browser.html
    soup = BeautifulSoup(browser.html, "html.parser")
    jobs = soup.find_all('div', {'class': 'cardOutline'})
    
    
    for job in jobs:
        title = job.find('a', {"class": 'jcs-JobTitle'}).text
        link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]

        company_name = job.find('span', {'class': 'companyName'})
        if company_name: 
            company = company_name.text
        else:
            company = None
        
        company_con = job.find('a', {"class": 'companyOverviewLink'})
        if company_con: 
            company_link = "www.indeed.com" + company_con["href"]
        else:
            company_link = None

        rating_con = job.find('span', {"class": 'ratingNumber'})
        if rating_con: 
            rating = rating_con.text
        else:
            rating = None


        loc = job.find('div', {"class": 'companyLocation'}).text

        salary_con = job.find('div', {"class": 'salary-snippet-container'})
        if salary_con: 
            salary = salary_con.text
        else:
            salary = None


        status_con = job.find('svg', {"aria-label": "Job type"})
        if status_con:
            status = status_con.parent.text
        else:
            status = None

        data = {}
        data["job_title"] = title
        data["job_link"] = link
        data["company"] = company
        data["company_link"] = company_link
        data["company_rating"] = rating
        data["location"] = loc
        data["salary"] = salary
        data["status"] = status
        rows.append(data)
    
    browser.links.find_by_text(f"{i+1}").click()
        
df = pd.DataFrame(rows)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status
0,Quality Data Analyst - Southern Region (MMC an...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dc...,MaineHealth - Corporate,www.indeed.com/cmp/Mainehealth,3.6,"Biddeford, ME 04005",None,Full-time
1,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cy...,Epitec,www.indeed.com/cmp/Epitec,3.5,"Chicago, IL",$65 - $70 an hour Employer est.,Full-time +1
2,IT Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B8...,ALABAMA PORT AUTHORITY,None,None,"Mobile, AL 36602 (North Industrial Area area)","$46,000 - $99,000 a year Employer est.",Full-time
3,Digital Content Data Analyst (Remote),www.indeed.com/rc/clk?jk=cbb02ebd1607703e&fcci...,Home Depot / THD,www.indeed.com/cmp/The-Home-Depot,3.7,"Remote in Atlanta, GA 30301","$130,000 a year Employer est.",None
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,Full-time


In [8]:
len(df)

735

In [9]:
df["company_rating"] = pd.to_numeric(df.company_rating)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    522 non-null    object 
 4   company_rating  522 non-null    float64
 5   location        735 non-null    object 
 6   salary          349 non-null    object 
 7   status          562 non-null    object 
dtypes: float64(1), object(7)
memory usage: 46.1+ KB


In [10]:
#combine with regex s=space d=catch or match all numbers \+ = plus sign matches plus sign, regex meaning.
df["status"] = df.status.str.replace("\s\+\d","")

/var/folders/g8/dnbf824j671c2bqm2ld6k3b80000gn/T/ipykernel_5831/152645899.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["status"] = df.status.str.replace("\s\+\d","")


In [11]:
df.status.value_counts()

Full-time     476
Contract       77
Part-time       4
Temporary       3
Internship      1
Seasonal        1
Name: status, dtype: int64

In [12]:
#Status column -  ID, name, last_updated
df.status.unique()

array(['Full-time', None, 'Contract', 'Internship', 'Temporary',
       'Part-time', 'Seasonal'], dtype=object)

In [13]:
#Status lookup table
status = df.status.dropna().unique()
status_df = pd.DataFrame()
status_df["status"] = status
status_df

,status
0,Full-time
1,Contract
2,Internship
3,Temporary
4,Part-time
5,Seasonal


In [14]:
#Company lookup table, revised to drop the 2nd Insight Global below
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

Insight Global                      2
MaineHealth - Corporate             1
MIT                                 1
United States Steel                 1
TechnologyAdvice                    1
                                   ..
Aries Solutions                     1
PCS Global Tech                     1
Substack                            1
New Directions Behavioral Health    1
Lincoln Financial                   1
Name: company, Length: 641, dtype: int64

In [15]:
companies.loc[companies.company == "Insight Global"]

,company,company_link,company_rating
196,Insight Global,www.indeed.com/cmp/Insight-Global,3.9
276,Insight Global,None,NaN


In [16]:
df.loc[df.company == "Insight Global", "company_link"] = "www.indeed.com/cmp/Insight-Global"
df.loc[df.company == "Insight Global", "company_rating"] = 3.9

In [17]:
companies = df.loc[:,["company","company_link", "company_rating"]].drop_duplicates().reset_index(drop=True)
companies.company.value_counts()

MaineHealth - Corporate             1
Johnson & Johnson                   1
United States Steel                 1
TechnologyAdvice                    1
Agama Solutions                     1
                                   ..
Aries Solutions                     1
PCS Global Tech                     1
Substack                            1
New Directions Behavioral Health    1
Lincoln Financial                   1
Name: company, Length: 641, dtype: int64

In [18]:
#Companies lookup table
companies

,company,company_link,company_rating
0,MaineHealth - Corporate,www.indeed.com/cmp/Mainehealth,3.6
1,Epitec,www.indeed.com/cmp/Epitec,3.5
2,ALABAMA PORT AUTHORITY,None,NaN
3,Home Depot / THD,www.indeed.com/cmp/The-Home-Depot,3.7
4,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0
...,...,...,...
637,Markel Corporation,www.indeed.com/cmp/Markel-Corporation,3.6
638,Virginia Dept of Behavioral Health & Developme...,None,NaN
639,South State Bank,www.indeed.com/cmp/South-State-Bank,3.3
640,"University of California, Davis",www.indeed.com/cmp/Uc-Davis,4.2


In [19]:
#State 
def getState(loc):
    if "," in loc:
        state = loc.replace("\xa0", " ").split(",")[1].strip().split(" ")[0].strip()
    else:
        state = None
        
    return(state)

In [20]:
#Find the locations with"Downtown area"
getState("Austin, TX (Downtown area)")

'TX'

In [21]:
#
df["state"] = df.location.apply(lambda x: getState(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    523 non-null    object 
 4   company_rating  523 non-null    float64
 5   location        735 non-null    object 
 6   salary          349 non-null    object 
 7   status          562 non-null    object 
 8   state           512 non-null    object 
dtypes: float64(1), object(8)
memory usage: 51.8+ KB


In [22]:
#
df.state.value_counts()

TX    173
CA     67
NY     58
IL     21
VA     20
GA     19
CO     14
WA     12
FL     11
NC      9
PA      9
MN      9
AZ      7
WI      7
MA      7
MD      6
OK      6
UT      5
OH      4
TN      4
DC      4
OR      4
CT      4
IA      3
MO      3
NE      2
MS      2
AL      2
NJ      2
MI      2
DE      2
NM      2
PR      2
KS      1
LA      1
ME      1
VT      1
SD      1
NV      1
KY      1
SC      1
NH      1
RI      1
Name: state, dtype: int64

State look up table

In [23]:
#State look-up table
state = df.state.dropna().unique()
state_df = pd.DataFrame()
state_df["state"] = state
state_df

,state
0,ME
1,IL
2,AL
3,GA
4,TX
5,NY
6,CA
7,MN
8,WA
9,MD


In [24]:
#
def getRemote(loc):
    if "remote" in loc.lower():
        return(True)
    else:
        return(False)

In [25]:
#
df["is_remote"] = df.location.apply(lambda x: getRemote(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    523 non-null    object 
 4   company_rating  523 non-null    float64
 5   location        735 non-null    object 
 6   salary          349 non-null    object 
 7   status          562 non-null    object 
 8   state           512 non-null    object 
 9   is_remote       735 non-null    bool   
dtypes: bool(1), float64(1), object(8)
memory usage: 52.5+ KB


In [26]:
df.is_remote.value_counts()

True     409
False    326
Name: is_remote, dtype: int64

In [27]:
df.is_remote.info()

<class 'pandas.core.series.Series'>
RangeIndex: 735 entries, 0 to 734
Series name: is_remote
Non-Null Count  Dtype
--------------  -----
735 non-null    bool 
dtypes: bool(1)
memory usage: 863.0 bytes


In [28]:
is_remote = df.is_remote.dropna().unique()
is_remote_df = pd.DataFrame()
is_remote_df["is_remote"] = is_remote
is_remote_df

,is_remote
0,False
1,True


In [29]:
#Find the companies paying for ads
def getAd(link):
    if "pagead" in link.lower():
        return(True)
    else:
        return(False)

In [30]:
df["is_ad"] = df.job_link.apply(lambda x: getAd(x))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735 entries, 0 to 734
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job_title       735 non-null    object 
 1   job_link        735 non-null    object 
 2   company         733 non-null    object 
 3   company_link    523 non-null    object 
 4   company_rating  523 non-null    float64
 5   location        735 non-null    object 
 6   salary          349 non-null    object 
 7   status          562 non-null    object 
 8   state           512 non-null    object 
 9   is_remote       735 non-null    bool   
 10  is_ad           735 non-null    bool   
dtypes: bool(2), float64(1), object(8)
memory usage: 53.2+ KB


In [31]:
df.is_ad.value_counts()

False    627
True     108
Name: is_ad, dtype: int64

In [32]:
#Paying for Ads, add to main job table as bool
df.groupby("company").is_ad.sum().sort_values(ascending=False)

company
Insight Global                      4
MATRIX Resources                    3
Michael Page                        3
Computer Enterprises, Inc. (CEI)    2
Presbyterian Healthcare Services    2
                                   ..
GENUENT                             0
GOIN                                0
GPRS                                0
Gainwell Technologies LLC           0
sgsco                               0
Name: is_ad, Length: 641, dtype: int64

In [33]:
import pandas as pd
from sqlalchemy import create_engine, inspect

In [44]:
USERNAME = "postgres"
PASSWORD = "password"
URL = "localhost"
PORT = 5432
DATABASE = "Project2_job_board"

connection_str = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{URL}:{PORT}/{DATABASE}"
#print(connection_str)

engine = create_engine(connection_str)

In [45]:
# create the inspector and connect to the engine
inspector_gadget = inspect(engine)

tables = inspector_gadget.get_table_names()
for table in tables:
    print(table)
    
    # get all columns in table
    columns = inspector_gadget.get_columns(table)
    for column in columns:
        print(column)
    print()

company
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('company_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'company', 'type': VARCHAR(length=100), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'company_link', 'type': VARCHAR(length=500), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'company_rating', 'type': DOUBLE_PRECISION(precision=53), 'nullable': True, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}

job
{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('job_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'job_title', 'type': VARCHAR(length=100), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'job_link', 'type': VARCHAR(length

In [46]:
status_df.head()

,status
0,Full-time
1,Contract
2,Internship
3,Temporary
4,Part-time


In [47]:
engine.execute("truncate table status restart identity cascade")
status_df.to_sql("status", engine, method="multi", if_exists = "append", index=False)

6

In [48]:
engine.execute("truncate table state restart identity cascade")
state_df.to_sql("state", engine, method="multi", if_exists = "append", index=False)

43

In [49]:
#Drop one company with no name, inclue in writeup
companies = companies.loc[pd.notnull(companies.company)].reset_index(drop=True)
companies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641 entries, 0 to 640
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company         641 non-null    object 
 1   company_link    443 non-null    object 
 2   company_rating  443 non-null    float64
dtypes: float64(1), object(2)
memory usage: 15.1+ KB


In [50]:
engine.execute("truncate table company restart identity cascade")
companies.to_sql("company", engine, method="multi", if_exists = "append", index=False)

641

In [51]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,status,state,is_remote,is_ad
0,Quality Data Analyst - Southern Region (MMC an...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dc...,MaineHealth - Corporate,www.indeed.com/cmp/Mainehealth,3.6,"Biddeford, ME 04005",None,Full-time,ME,False,True
1,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cy...,Epitec,www.indeed.com/cmp/Epitec,3.5,"Chicago, IL",$65 - $70 an hour Employer est.,Full-time,IL,False,True
2,IT Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B8...,ALABAMA PORT AUTHORITY,None,NaN,"Mobile, AL 36602 (North Industrial Area area)","$46,000 - $99,000 a year Employer est.",Full-time,AL,False,True
3,Digital Content Data Analyst (Remote),www.indeed.com/rc/clk?jk=cbb02ebd1607703e&fcci...,Home Depot / THD,www.indeed.com/cmp/The-Home-Depot,3.7,"Remote in Atlanta, GA 30301","$130,000 a year Employer est.",None,GA,True,False
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,Full-time,IL,False,True


In [52]:
status2 = pd.read_sql("select id as status_id, status from status", engine)
status2.head(20)

,status_id,status
0,1,Full-time
1,2,Contract
2,3,Internship
3,4,Temporary
4,5,Part-time
5,6,Seasonal


In [53]:
df = pd.merge(df, status2, on = "status", how ="left")
df.drop("status", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,state,is_remote,is_ad,status_id
0,Quality Data Analyst - Southern Region (MMC an...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dc...,MaineHealth - Corporate,www.indeed.com/cmp/Mainehealth,3.6,"Biddeford, ME 04005",None,ME,False,True,1.0
1,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cy...,Epitec,www.indeed.com/cmp/Epitec,3.5,"Chicago, IL",$65 - $70 an hour Employer est.,IL,False,True,1.0
2,IT Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B8...,ALABAMA PORT AUTHORITY,None,NaN,"Mobile, AL 36602 (North Industrial Area area)","$46,000 - $99,000 a year Employer est.",AL,False,True,1.0
3,Digital Content Data Analyst (Remote),www.indeed.com/rc/clk?jk=cbb02ebd1607703e&fcci...,Home Depot / THD,www.indeed.com/cmp/The-Home-Depot,3.7,"Remote in Atlanta, GA 30301","$130,000 a year Employer est.",GA,True,False,NaN
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,IL,False,True,1.0


In [54]:
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,state,is_remote,is_ad,status_id
0,Quality Data Analyst - Southern Region (MMC an...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dc...,MaineHealth - Corporate,www.indeed.com/cmp/Mainehealth,3.6,"Biddeford, ME 04005",None,ME,False,True,1.0
1,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cy...,Epitec,www.indeed.com/cmp/Epitec,3.5,"Chicago, IL",$65 - $70 an hour Employer est.,IL,False,True,1.0
2,IT Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B8...,ALABAMA PORT AUTHORITY,None,NaN,"Mobile, AL 36602 (North Industrial Area area)","$46,000 - $99,000 a year Employer est.",AL,False,True,1.0
3,Digital Content Data Analyst (Remote),www.indeed.com/rc/clk?jk=cbb02ebd1607703e&fcci...,Home Depot / THD,www.indeed.com/cmp/The-Home-Depot,3.7,"Remote in Atlanta, GA 30301","$130,000 a year Employer est.",GA,True,False,NaN
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,IL,False,True,1.0


In [55]:
state2 = pd.read_sql("select id as state_id, state from state", engine)
df = pd.merge(df, state2, on = "state", how ="left")
df.drop("state", axis=1, inplace=True)
df.head()

,job_title,job_link,company,company_link,company_rating,location,salary,is_remote,is_ad,status_id,state_id
0,Quality Data Analyst - Southern Region (MMC an...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dc...,MaineHealth - Corporate,www.indeed.com/cmp/Mainehealth,3.6,"Biddeford, ME 04005",None,False,True,1.0,1.0
1,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cy...,Epitec,www.indeed.com/cmp/Epitec,3.5,"Chicago, IL",$65 - $70 an hour Employer est.,False,True,1.0,2.0
2,IT Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B8...,ALABAMA PORT AUTHORITY,None,NaN,"Mobile, AL 36602 (North Industrial Area area)","$46,000 - $99,000 a year Employer est.",False,True,1.0,3.0
3,Digital Content Data Analyst (Remote),www.indeed.com/rc/clk?jk=cbb02ebd1607703e&fcci...,Home Depot / THD,www.indeed.com/cmp/The-Home-Depot,3.7,"Remote in Atlanta, GA 30301","$130,000 a year Employer est.",True,False,NaN,4.0
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,Lettuce Entertain You Restaurants (Chicago),www.indeed.com/cmp/Lettuce-Entertain-You,4.0,"Chicago, IL 60631 (Norwood Park area)",None,False,True,1.0,2.0


In [56]:
company2 = pd.read_sql("select id as company_id, company from company", engine)
df = pd.merge(df, company2, on = "company", how ="left")
df.drop(["company", "company_link", "company_rating"], axis=1, inplace=True)
df.head()

,job_title,job_link,location,salary,is_remote,is_ad,status_id,state_id,company_id
0,Quality Data Analyst - Southern Region (MMC an...,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Dc...,"Biddeford, ME 04005",None,False,True,1.0,1.0,1.0
1,Data Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0Cy...,"Chicago, IL",$65 - $70 an hour Employer est.,False,True,1.0,2.0,2.0
2,IT Business Analyst,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B8...,"Mobile, AL 36602 (North Industrial Area area)","$46,000 - $99,000 a year Employer est.",False,True,1.0,3.0,3.0
3,Digital Content Data Analyst (Remote),www.indeed.com/rc/clk?jk=cbb02ebd1607703e&fcci...,"Remote in Atlanta, GA 30301","$130,000 a year Employer est.",True,False,NaN,4.0,4.0
4,Operating Systems Analyst - Corporate Office,www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0B1...,"Chicago, IL 60631 (Norwood Park area)",None,False,True,1.0,2.0,5.0


In [57]:
#Database
df.to_sql("job", engine, method="multi", if_exists = "append", index=False)

735

Test Code & Trials

In [ ]:
#Find the jobs titles for positions and the link IF available
#job = jobs[0]
#title = job.find('a', {"class": 'jcs-JobTitle'}).text
#link = "www.indeed.com" + job.find('a', {"class": 'jcs-JobTitle'})["href"]

In [ ]:
#Find company name and the link IF available
#company = job.find('a', {"class": 'companyOverviewLink'}).text
#company_link = "www.indeed.com" + job.find('a', {"class": 'companyOverviewLink'})["href"]

In [ ]:
#Find the rating for the company/job
#rating = job.find('span', {"class": 'ratingNumber'}).text
#rating

In [ ]:
#Find the location of the job listing
#loc = job.find('div', {"class": 'companyLocation'}).text
#loc

In [ ]:
#Find the salary for the job listing
#salary = job.find('div', {"class": 'salary-snippet-container'}).text
#salary

In [ ]:
#Pull the text only from the parent for job salary
#job.find('svg', {"aria-label": "Job type"}).parent.text

In [ ]:
#Don't worry about it
# def getCity(loc):
#     if "," in loc:
#         city_list = loc.replace("\xa0", " ").split(",")[0].strip().split(" ")
#         city = city_list[0] if len(city_list) ==1 else city_list[1]
#     else:
#         city = None
        
#     return(city)

In [ ]:
# df["city"] = df.location.apply(lambda x: getCity(x))
# df.info()

In [ ]:
# df.city.value_counts()

In [ ]:
# df.loc[df.location.str.contains("in")]

In [ ]:
#Create a CSV for companies 
#companies.to_csv('companies.csv', index=False)

In [ ]:
#companies_file = pd.read_csv('companies.csv')
#companies_file

In [ ]:
#Create a CSV for States
#state_df.to_csv('state.csv', index=False)

In [ ]:
#state_file = pd.read_csv('state.csv')
#state_file

In [ ]:
#Create a CSV for status (Full-time/part-time)
#status_df.to_csv('status.csv', index=False)

In [ ]:
#status_file = pd.read_csv('status.csv')
#status_file

In [ ]:
#Set-up sleep function if needed
# browser.visit(dice_lyst)
# time.sleep(2)